In [2]:
%load_ext blackcellmagic

In [118]:
from flask import Flask, request, render_template, jsonify, json
from decouple import config
from flask_cors import CORS
import numpy as np
import networkx as nx
import tweepy
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re
from collections import Counter
import asyncio
from concurrent.futures import ThreadPoolExecutor
import requests
from itertools import chain
import nest_asyncio


def get_network(user):
    headers = {"Content-Type": "application/json"}
    body = {
        "TWITTER_ACCESS_TOKEN": config("TWITTER_ACCESS_TOKEN"),
        "TWITTER_ACCESS_TOKEN_SECRET": config("TWITTER_ACCESS_TOKEN_SECRET"),
        "search": user,
    }
    edge_list = requests.post(
        url="https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
        headers=headers,
        data=json.dumps(body),
    )
    edge_list=edge_list.json()
    edge_df=pd.DataFrame(edge_list,columns=["source","target"])
    # create a graph and add edges
    DG = nx.MultiGraph()
    DG = nx.from_pandas_edgelist(edge_df, "source", "target")
    DG_layout = nx.spectral_layout(DG)
    
    # create json nodes from graph
    sl_nodes = nx.spectral_layout(DG)
    node_list = list(sl_nodes)
    node_x = [sl_nodes[i][0] for i in node_list]
    node_y = [sl_nodes[i][1] for i in node_list]
    node_x = json.dumps(node_x)
    node_y = json.dumps(node_y)
    
    node_counter = Counter(list(chain.from_iterable(edge_list)))
    node_keys = list(sl_nodes.keys())
    node_weights = [node_counter[i] for i in node_keys]
    node_weights = json.dumps(node_weights)
    node_list = json.dumps(node_list)

    # create json edges from graph
    edges = list(DG.edges.data())
    sl_edges_x = []
    sl_edges_y = []
    sl_edges_weight = []
    for i in range(len(edges)):
        sl_edges_x.append(sl_nodes[edges[i][0]][0])
        sl_edges_y.append(sl_nodes[edges[i][0]][1])
        sl_edges_x.append(sl_nodes[edges[i][1]][0])
        sl_edges_y.append(sl_nodes[edges[i][1]][1])
        sl_edges_x.append("")
        sl_edges_y.append("")
        sl_edges_weight.append("")
    x = json.dumps(sl_edges_x)
    y = json.dumps(sl_edges_y)
    weights = json.dumps(sl_edges_weight)

    user = json.dumps(user)

    #         return render_template("network.html", edge_x =x, edge_y =y, node_x =
    #         node_x, node_y =node_y, user = user, node_list=node_list,
    #         node_weights=node_weights)
    return x, y, node_x, node_y, user, node_list, node_weights

In [119]:
get_network("austen")

('[0.11541499065995, 0.12878509062134647, "", 0.11541499065995, 0.1287850906213464, "", 0.11541499065995, 0.1328938255085541, "", 0.11541499065995, -0.12917863638085045, "", 0.11541499065995, 0.11875288190900048, "", 0.11541499065995, 0.08258988132760506, "", 0.11541499065995, 0.12878509062134635, "", 0.11541499065995, -0.0177382279878134, "", 0.11541499065995, 0.12878509062134655, "", 0.11541499065995, 0.12878509062134644, "", 0.11541499065995, 0.12878509062134647, "", 0.11541499065995, 0.1287850906213465, "", 0.11541499065995, 0.0182119889691456, "", 0.11541499065995, 0.13657600328506062, "", 0.11541499065995, 0.1287850906213464, "", 0.11541499065995, 0.13289382550855425, "", 0.11541499065995, 0.1287850906213464, "", 0.11541499065995, -0.04811362021202532, "", 0.11541499065995, 0.12878509062134644, "", 0.11541499065995, 0.1287850906213463, "", 0.11541499065995, 0.12878509062134644, "", 0.11541499065995, 0.12878509062134647, "", 0.11541499065995, 0.1287850906213465, "", 0.115414990659

In [116]:
user="austen"
headers = {"Content-Type": "application/json"}
body = {
    "TWITTER_ACCESS_TOKEN": config("TWITTER_ACCESS_TOKEN"),
    "TWITTER_ACCESS_TOKEN_SECRET": config("TWITTER_ACCESS_TOKEN_SECRET"),
    "search": user,
}
edge_list = requests.post(
    url="https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
    headers=headers,
    data=json.dumps(body),
)

In [117]:
edge_list

<Response [500]>

In [100]:
edge_list = edge_list.json()


In [109]:
edge_df=pd.DataFrame(edge_list,columns=["source","target"])

In [111]:
edge_df

,source,target
0,austen,FFNinja3
1,austen,LambdaSchool
2,austen,tommycollison
3,austen,nullagent
4,austen,AMartinCastro
5,austen,jayvasdigital
6,austen,tommycollison
7,austen,LambdaSchool
8,austen,jayvasdigital
9,austen,AMartinCastro


In [83]:
DG = nx.MultiGraph()
DG = nx.from_pandas_edgelist(edge_df, 0, 1)

In [84]:
nx.draw_spectral(DG)

In [28]:
sl_nodes = nx.spectral_layout(DG)
node_list = list(sl_nodes)

In [29]:
node_x = [sl_nodes[i][0] for i in node_list]
node_y = [sl_nodes[i][1] for i in node_list]
node_x = json.dumps(node_x)
node_y = json.dumps(node_y)

In [34]:
from itertools import chain
node_list = Counter(list(chain.from_iterable(edge_list)))

In [61]:
node_list["austen"]

5105

[['austen', 'sarahmceneaney'],
 ['austen', 'patio11'],
 ['austen', 'rrherr'],
 ['austen', 'innov8tor3'],
 ['austen', 'Dave_Wolf'],
 ['austen', 'LambdaSchool'],
 ['austen', 'innov8tor3'],
 ['austen', 'Dave_Wolf'],
 ['austen', 'LambdaSchool'],
 ['austen', 'dnsbty'],
 ['austen', 'imchrislouie'],
 ['austen', 'sarthakgh'],
 ['austen', 'zck'],
 ['austen', 'marcosmartig'],
 ['austen', 'LambdaSchool'],
 ['austen', 'LambdaSchool'],
 ['austen', 'SanFernandinho'],
 ['austen', 'Austen'],
 ['austen', 'nlsanders'],
 ['austen', 'CoderCrypto'],
 ['austen', 'danrhowarth'],
 ['austen', 'SamSamskies'],
 ['austen', 'aminafoon'],
 ['austen', 'SchneiderKJ'],
 ['austen', 'RyanNegri'],
 ['austen', 'justGLew'],
 ['austen', 'N_Sportelli'],
 ['austen', 'tylerwillis'],
 ['austen', 'stratechery'],
 ['austen', 'GraehamF'],
 ['austen', 'eraldocavalli'],
 ['austen', 'cyppan'],
 ['austen', 'MikeIsaac'],
 ['austen', 'achalaugustine'],
 ['austen', 'danielgross'],
 ['austen', 'achalaugustine'],
 ['austen', 'danielgross']

In [55]:
node_keys = list(sl_nodes.keys())

In [76]:
node_weights = [node_list[i] for i in node_keys]